In [1]:
import numpy as np
import pandas as pd
import glob
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report, make_scorer, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import KFold
import ngram
import random
import os

In [2]:
def tokenize(word):
    word_list = []
    for n in range(2, 4):
        index = ngram.NGram(N=n)
        for w in index.ngrams(index.pad(word)):
            if w.find("$") != -1:
                continue
            word_list.append(w)
    return word_list

In [3]:
train_df = pd.read_csv("./train.csv")
test_df = pd.read_csv("./test.csv")
compe_df = pd.read_csv("./compe.csv")

In [4]:
compe_df

,text
0,電車がまだ動かなくて自宅待機中\n家庭科の先生ほんと嫌いやから、4時間目まで動かないで欲しい...
1,TSUTAYAに行って、作品選んで、でもいざレジに行こうとすると動悸がして、結局トイレットペ...
2,ほっといてよもうほんとにまじで何妥協とか遠慮とか知らないの？？？？？お前ほんと人間？？？今ま...
3,朝早く起きすぎた
4,どんな授業を受けても、どんな講演を受けても、何も感じられない。そんな私にとって、コメントペー...
...,...
9995,@wyvern4034 お疲れ様です😆
9996,そんな君が私の玩具
9997,川崎市の40~59歳は、まだ、予約サイトから予約できないよ。
9998,@ayaka_1999_5 犬嶋だよ！！に見えた((


In [5]:
train_text = train_df["text"].values.astype('U')
test_text = test_df["text"].values.astype('U')
y = train_df["label"].values.astype("int8")
test_y = test_df["label"].values.astype("int8")
compe_text = compe_df["text"].values.astype('U')

In [6]:
vectorizer = CountVectorizer(analyzer=tokenize)
vectorizer.fit(train_text)
X = vectorizer.transform(train_text)
test_vec = vectorizer.transform(test_text)
compe_vec = vectorizer.transform(compe_text)

In [7]:
valid_scores = []

train_check = np.zeros(len(y), dtype=np.float64)
models = []

kf = KFold(n_splits=3)
for fold, (tr_idx, va_idx) in enumerate(kf.split(X, y)):
    x_train, y_train = X[tr_idx], y[tr_idx]
    x_valid, y_valid = X[va_idx], y[va_idx]
    m = LinearRegression()
    m.fit(x_train, y_train)
    p = m.predict(x_valid)
    train_check[va_idx] += p
    models.append(m)

accuracy_score(y, np.where(train_check<0.5, 0, 1))

0.7383

In [8]:
result = np.zeros(len(test_y), dtype=np.float64)
result = np.array([model.predict(test_vec) for model in models]).mean(axis=0)
result = np.where(result<0.5, 0, 1)

In [9]:
accuracy_score(test_y, result) #リーダーボードのスコア この値を毎回，discord上に載せてください．

0.769

In [10]:
len(result)

1000

In [11]:
c_result = np.zeros(len(compe_text), dtype=np.float64)
c_result = np.array([model.predict(compe_vec) for model in models]).mean(axis=0)
c_result = np.where(c_result<0.5, 0, 1)
len(c_result)

10000

In [12]:
csv_data = pd.DataFrame(data=c_result, columns=["label"])
csv_data.reset_index(inplace=True)
csv_data = csv_data.rename(columns={'index': 'ID'})
csv_data.to_csv("./submission.csv", index=False) #submission.csvを最大2つまで馬場宛に提出してください．良い方の結果を最終スコアとします．